In [1]:
import keras

Using TensorFlow backend.


In [2]:
TWEETS_DIR = './obama-trump-data/tweets'
LABELS_DIR = './obama-trump-data/labels_np'

#word embeddings file directory 
EMBEDDINGS_DIR = 'glove.6B.50d.txt'

In [3]:
import pickle

# Load tweets, labels
tweets = pickle.load(open(TWEETS_DIR,'rb'))
labels = pickle.load(open(LABELS_DIR,'rb'))

# Sample tweets
for i in range(0,100,10):
    print("Tweet: ", tweets[i], ". Label: ", labels[i])

Tweet:  "America should be very proud." President Obama #LoveWins . Label:  0
Tweet:  TO ALL AMERICANS
https://t.co/D7Es6ie4fY . Label:  1
Tweet:  No deal was made last night on DACA. Massive border security would have to be agreed to in exchange for consent. Would be subject to vote. . Label:  1
Tweet:  I was viciously attacked by Mr. Khan at the Democratic Convention. Am I not allowed to respond? Hillary voted for the Iraq war, not me! . Label:  1
Tweet:  .@RudyGiuliani, one of the finest people I know and a former GREAT Mayor of N.Y.C., just took himself out of consideration for "State". . Label:  1
Tweet:  The #TPP establishes the highest labor standards of any trade agreement in history. http://t.co/3vqoancyYp . Label:  0
Tweet:  Thoughts and prayers to the great people of Indiana. You will prevail! . Label:  1
Tweet:  Hillary said she was under sniper fire (while surrounded by USSS.) Turned out to be a total lie. She is not fit to https://t.co/hBIrGj21l6 . Label:  1
Tweet:  The f

In [4]:
import numpy as np
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenize the tweets (convert sentence to sequence of words)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tweets)

sequences = tokenizer.texts_to_sequences(tweets)
word_index = tokenizer.word_index

print('Found %s unique tokens (words).' % len(word_index))

# Pad sequences to ensure samples are the same size
training_data = pad_sequences(sequences)

print("Training data size is (%d,%d)"  % training_data.shape) # shape = (num. tweets, max. length of tweet)
print("Labels are size %d"  % labels.shape)

Found 13049 unique tokens (words).
Training data size is (6136,31)
Labels are size 6136


In [5]:
# Show effect of tokenization, padding
print("Original tweet: ", tweets[0])
print("Tweet after tokenization and padding: ", training_data[0])

Original tweet:  "America should be very proud." President Obama #LoveWins
Tweet after tokenization and padding:  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0  44  88  21  83 391  12  14 794]


In [6]:
# Convert words to word embedding vectors

EMBEDDING_DIM = 50
print('Indexing word vectors.')

import os
embeddings_index = {}
f = open(EMBEDDINGS_DIR, 'rb') # NOTE: if using Windows and getting errors: change this to open(EMBEDDINGS_DIR, 'rb')
for line in f:
    values = line.split()
    word = values[0].decode('UTF-8') # NOTE: if you made that 'rb' change: add to this line, values[0].decode('UTF-8')
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# prepare word embedding matrix
num_words = len(word_index)+1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Indexing word vectors.
Found 400000 word vectors.


In [7]:
# Sample word embedding vector:
print(word_index["computer"]) # retrieve word index
print(embedding_matrix[2645]) # use word index to retrieve word embedding vector

2645
[ 0.079084   -0.81503999  1.79009998  0.91653001  0.10797    -0.55628002
 -0.84426999 -1.49510002  0.13417999  0.63626999  0.35146001  0.25813001
 -0.55028999  0.51055998  0.37408999  0.12092    -1.61660004  0.83653003
  0.14202    -0.52348     0.73452997  0.12207    -0.49079001  0.32532999
  0.45306    -1.58500004 -0.63848001 -1.00530005  0.10454    -0.42984
  3.18099999 -0.62186998  0.16819    -1.01390004  0.064058    0.57844001
 -0.45559999  0.73782998  0.37202999 -0.57722002  0.66441     0.055129
  0.037891    1.32749999  0.30991     0.50696999  1.23570001  0.1274
 -0.11434     0.20709001]


In [8]:
from keras.models import Sequential
from keras.layers import Embedding, Input
from keras.layers.merge import Concatenate
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Dropout, concatenate
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras import metrics
from keras.models import Model
import pickle

In [9]:
model = Sequential()

# Add pre-trained embedding layer 
    # converts word indices to GloVe word embedding vectors as they're fed in
model.add(Embedding(len(word_index) + 1,
                    EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=training_data.shape[1],
                    trainable=False))

# At this point, each individual training sample is now a sequence of word embedding vectors

# First LSTM layer (return sequence so that we can feed the output into the 2nd LSTM layer)
model.add(LSTM(128, return_sequences = True, activation='relu'))
model.add(Dropout(.2))

# Second LSTM layer 
    # Don't return sequence this time, because we're feeding into a fully-connected layer
model.add(LSTM(128, activation='relu'))
model.add(Dropout(.2))

# Dense 1
model.add(Dense(32, activation='relu'))
model.add(Dropout(.2))

# Dense 2 (final vote)
model.add(Dense(1, activation = 'sigmoid'))

print(model.summary())

LOSS = 'binary_crossentropy' # because we're classifying between 0 and 1
OPTIMIZER = 'RMSprop' # RMSprop tends to work well for recurrent models

model.compile(loss = LOSS, optimizer = OPTIMIZER, metrics = [metrics.binary_accuracy])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 31, 50)            652500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 31, 128)           91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
dro

In [10]:
EPOCHS = 8
BATCH_SIZE = 150

model.fit(training_data, labels, 
          epochs = EPOCHS, 
          batch_size = BATCH_SIZE, 
          validation_split =.2)

Train on 4908 samples, validate on 1228 samples
Epoch 1/8
4908/4908 [==============================] - 6s 1ms/step - loss: 0.5701 - binary_accuracy: 0.6899 - val_loss: 0.3661 - val_binary_accuracy: 0.8396
Epoch 2/8
4908/4908 [==============================] - 4s 900us/step - loss: 0.3764 - binary_accuracy: 0.8443 - val_loss: 0.4529 - val_binary_accuracy: 0.7801
Epoch 3/8
4908/4908 [==============================] - 5s 976us/step - loss: 0.3077 - binary_accuracy: 0.8753 - val_loss: 0.2398 - val_binary_accuracy: 0.9031
Epoch 4/8
4908/4908 [==============================] - 4s 906us/step - loss: 0.2415 - binary_accuracy: 0.9040 - val_loss: 0.2142 - val_binary_accuracy: 0.9121
Epoch 5/8
4908/4908 [==============================] - 4s 895us/step - loss: 0.2344 - binary_accuracy: 0.9097 - val_loss: 0.1892 - val_binary_accuracy: 0.9186
Epoch 6/8
4908/4908 [==============================] - 4s 889us/step - loss: 0.1896 - binary_accuracy: 0.9209 - val_loss: 0.3058 - val_binary_accuracy: 0.8762


In [11]:
score, acc = model.evaluate(training_data, labels,
                            batch_size=BATCH_SIZE)
print('Test score:', score)
print('Test accuracy:', acc)

6136/6136 [==============================] - 2s 295us/step
Test score: 0.16587420085486332
Test accuracy: 0.9341590602480293
